You arrive at the Venus fuel depot only to discover it's protected by a password. The Elves had written the password on a sticky note, but someone threw it out.

However, they do remember a few key facts about the password:

- It is a six-digit number.
- The value is within the range given in your puzzle input.
- Two adjacent digits are the same (like 22 in 122345).
- Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).

Other than the range rule, the following are true:

111111 meets these criteria (double 11, never decreases).
223450 does not meet these criteria (decreasing pair of digits 50).
123789 does not meet these criteria (no double).
How many different passwords within the range given in your puzzle input meet these criteria?

#### Your puzzle input is `307237-769058`.

In [1]:
INPUT_LOW, INPUT_HIGH = 307237, 769058

### Part 1

In [2]:
%%time
import z3

s = z3.Solver()

# six digit number
d = [z3.Int(f"{i}") for i in range(6)]
for digit in d:
    s.add(digit >= 0, digit <= 9)

# in range
val = d[0] * 10**5 + d[1] * 10**4 + d[2] * 10**3 + d[3] * 10**2 + d[4] * 10**1 + d[5] * 10**0
s.add(val >= INPUT_LOW)
s.add(val <= INPUT_HIGH)

# two adjacent are the same
adj = [d[i] == d[i+1] for i in range(5)]
s.add(z3.Or(adj))

# never decrease
adj = [d[i+1] >= d[i] for i in range(5)]
s.add(z3.And(adj))

solutions = 0
while True:
    if s.check() != z3.sat:
        break
    else:
        m = s.model()
        answer = int("".join([str(m[digit]) for digit in d]))
        solutions += 1
        s.add(val != answer)
        if solutions % 100 == 0:
            print(solutions, "->", answer)

100 -> 556688
200 -> 466789
300 -> 333488
400 -> 334456
500 -> 367779
600 -> 344788
700 -> 455568
800 -> 447888
CPU times: user 10.8 s, sys: 12.9 ms, total: 10.8 s
Wall time: 10.8 s


In [3]:
solutions

889

### Part 2

In [5]:
%%time
import z3

s = z3.Solver()

# six digit number
d = [z3.Int(f"{i}") for i in range(6)]
for digit in d:
    s.add(digit >= 0, digit <= 9)

# in range
val = d[0] * 10**5 + d[1] * 10**4 + d[2] * 10**3 + d[3] * 10**2 + d[4] * 10**1 + d[5] * 10**0
s.add(val >= INPUT_LOW)
s.add(val <= INPUT_HIGH)

# at least one where two adjacent are the same but not more than two
two_adjacent_possibilities = []
for i in range(5):
    conds = []
    # digit to the left not the same unless at leftmost digits
    if i > 0:
        conds.append(d[i-1] < d[i])
    # middle two are the same
    conds.append(d[i] == d[i+1])
    # digit to the right not the same unless at rightmost digits
    if i < (len(d) - 2):
        conds.append(d[i+1] < d[i+2])
    one_way_to_have_only_two_adjacent = z3.And(conds)
    two_adjacent_possibilities.append(one_way_to_have_only_two_adjacent)
all_ways_to_have_only_two_adjacent = z3.Or(two_adjacent_possibilities)
s.add(all_ways_to_have_only_two_adjacent)

# never decrease
adj = [d[i+1] >= d[i] for i in range(5)]
s.add(z3.And(adj))

solutions = 0
while True:
    if s.check() != z3.sat:
        break
    else:
        m = s.model()
        answer = int("".join([str(m[digit]) for digit in d]))
        solutions += 1
        s.add(val != answer)
        if solutions % 100 == 0:
            print(solutions, "->", answer)

100 -> 355779
200 -> 334469
300 -> 455588
400 -> 555799
500 -> 577788
CPU times: user 7.7 s, sys: 0 ns, total: 7.7 s
Wall time: 7.69 s


In [6]:
solutions

589